<a href="https://colab.research.google.com/github/onticonti/SEO_Python/blob/master/export_keyword_tocsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from requests.exceptions import Timeout
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
 
keyword = input("Arama yapılacak anahtar kelimeyi giriniz : ")
filename = keyword.replace(' ', '_')
 
serp = {
    'urls': [],
    'titles': [],
    'meta_desc': [],
    'h1': [],
    'h2': [],
    'h3': [],
    'b': [],
    'strong': [],
    'em': [],
    'i': [],
    'a': []
}
 
params = {
    'access_key': 'a7c264f20edb7061f30d3fa0e8ca7503',
    'query': keyword
}
 
api_result = requests.get('http://api.serpstack.com/search', params, verify=False)
 
api_response = api_result.json()
 
org = api_response.get("organic_results")
 
for results in range(0,len(org)):
    link = (org[results]['url'])
    serp['urls'].append(link)
    serp['titles'].append(org[results]['title'])
    serp['meta_desc'].append(org[results]['snippet'])
    try:
        page = requests.get(link, timeout=5)
    except Timeout:
        print("Timed out!")
    else:
        if page.status_code >= 400:
            continue
    soup = BeautifulSoup(page.content, "html.parser", from_encoding="iso-8859-1")
    for headings in soup.find_all(["h1", "h2", "h3", "h4", "b", "strong", "em", "i", "a"]):
        if headings.name == 'h1' and headings.text not in serp.keys():
            serp['h1'].append(headings.text.replace('\n',""))
        elif headings.name == 'h2' and headings.text not in serp.keys():
            serp['h2'].append(headings.text.replace('\n',""))
        elif headings.name == 'h3' and headings.text not in serp.keys():
            serp['h3'].append(headings.text.replace('\n',""))
        elif headings.name == 'b' and headings.text not in serp.keys():
            serp['b'].append(headings.text.replace('\n',""))
        elif headings.name == 'strong' and headings.text not in serp.keys():
            serp['strong'].append(headings.text.replace('\n',""))
        elif headings.name == 'em' and headings.text not in serp.keys():
            serp['em'].append(headings.text.replace('\n',""))
        elif headings.name == 'i' and headings.text not in serp.keys():
            serp['i'].append(headings.text.replace('\n',""))
        elif headings.name == 'a' and headings not in serp.keys():
            serp['a'].append(headings.get('href'))          
        
df = {key:pd.Series(value) for key, value in serp.items()}
 
serpdata = pd.DataFrame(df)
print(serpdata)
 
serpdata.to_csv(f'{filename}.csv', header=True, index=False,encoding="utf8")
files.download(f'{filename}.csv')